In [58]:
from Explanations_Models.Custom_DT.LIME import LIME
from FeatureImportance.FI import FeatureImportance
import yaml
import pandas as pd
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
import json
import warnings
from tqdm import tqdm
import torch
# Suppress all deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

yaml.add_constructor("!join", join)
parser = argparse.ArgumentParser()

parser.add_argument("--config_filename", required=False, type=str)

config_file = open("config_envs/{}.yml".format("cartpole"))
config = yaml.load(config_file, Loader=yaml.FullLoader)
config.update(yaml.load(open("config_explanations/{}.yml".format("cartpole"), encoding="utf8"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights(PATH = None)


def PCA_Ish(FI, out_logits):
    tmp = FI.mean()
    arr = []
    for i in out_logits.keys():
        arr.append(tmp[i].to_numpy())
    tmp = np.stack(arr)
    A = torch.tensor(tmp, dtype=torch.float64).T

    print(A)
    print(A.T@A)
    vals, vecs = torch.linalg.eig(A@A.T)
    #print(torch.linalg.eig(A))
    print(vals.real, vecs.real)
    weighted_FI = torch.unsqueeze(vals.real,1) * vecs.real
    return torch.sum(weighted_FI, dim=0).detach().numpy()




device:  cpu


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\EnvRunner.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.baseline_network.load_state_

In [59]:
DT = LIME(config, Runner)
X,Y = DT.sample_set()

FI = FeatureImportance(config, Runner.model.network) 
out, fi = FI.Relevence(X)

columns = pd.MultiIndex.from_product([config["picture"]["class_names"], config["picture"]["labels"]], names=['OutLogit', 'InLogit'])
fi = np.abs(fi)
# Reshape the data to (60, 8) to match the multi-level column structurel
reshaped_data = fi.reshape(fi.shape[0], -1)

# Create the DataFrame
df = pd.DataFrame(reshaped_data, columns=columns)
out = pd.DataFrame(out, columns=config["picture"]["class_names"])

In [60]:

PCA_Ish(df, out)

tensor([[0.2976, 0.4349],
        [0.2876, 0.2182],
        [0.2862, 0.2952],
        [0.3477, 0.3278]], dtype=torch.float64)
tensor([[0.3741, 0.3907],
        [0.3907, 0.4313]], dtype=torch.float64)
tensor([ 7.9441e-01,  1.1016e-02, -4.8574e-18,  3.2106e-17],
       dtype=torch.float64) tensor([[-0.5847,  0.7437,  0.3035,  0.0372],
        [-0.3990, -0.5914,  0.4943, -0.2168],
        [-0.4612, -0.0825, -0.8146, -0.6757],
        [-0.5350, -0.3007,  0.0020,  0.7036]], dtype=torch.float64)


array([-0.4689105 ,  0.58426421,  0.24653739,  0.02712519])

In [67]:
tp1 = torch.DoubleTensor([[70.1270, 39.3422, 43.8028, 68.3783],
        [75.9377, 28.0442, 40.4800, 69.6060]])*(1/100)
a = torch.mm( tp1.t(),tp1)

eigVals, eigVecs = torch.linalg.eigh(a)

print(eigVals.real)
print(eigVecs.real)

tensor([-1.7421e-16,  5.6252e-17,  8.6973e-03,  2.6010e+00],
       dtype=torch.float64)
tensor([[ 0.3320, -0.5302,  0.4455, -0.6404],
        [-0.0907, -0.4178, -0.8543, -0.2955],
        [ 0.6327,  0.6333, -0.2491, -0.3695],
        [-0.6937,  0.3785,  0.0978, -0.6050]], dtype=torch.float64)


In [68]:
torch.svd(tp1)

torch.return_types.svd(
U=tensor([[ 0.7074, -0.7068],
        [ 0.7068,  0.7074]], dtype=torch.float64),
S=tensor([1.6127, 0.0933], dtype=torch.float64),
V=tensor([[ 0.6404,  0.4455],
        [ 0.2955, -0.8543],
        [ 0.3695, -0.2491],
        [ 0.6050,  0.0978]], dtype=torch.float64))

left     1.586315
right    1.718487
dtype: float32

In [80]:
def Policy_Smoothing_MaxAvg_Weighting(FI, out_logits):
    tmp = []
    for i in list(out_logits.keys()):
        tmp.append(FI[i].mean().to_numpy())
    tmp = tmp[np.argmax(np.stack(tmp).mean(axis=1))]
    tmp = pd.DataFrame(np.expand_dims(tmp, axis=0), columns = list(FI[out_logits.keys()[0]].keys()))
    return tmp
i = Policy_Smoothing_MaxAvg_Weighting(df, out)

In [82]:
i["angle"].iloc[0]

0.29521686

In [5]:
shape = 2
array = np.random.uniform(low=-1, high=1, size=shape)
print(array)

[-0.44366953  0.21514174]
